In [1]:
from IPython.display import display, HTML,clear_output
# css = """
# .output {
# flex-direction: row;
# flex-wrap: wrap;


# }
# """

# HTML('<style>{}</style>'.format(css))

In [2]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [3]:
# !pip install pandasql

In [4]:
# !pip install lxml
!pip install --upgrade 'sqlalchemy<2.0'

The system cannot find the file specified.


In [5]:
from pandasql import sqldf
import requests
import pandas as pd
import time
from bs4 import BeautifulSoup

In [ ]:
pd.options.mode.copy_on_write = True

In [ ]:
def MC_strickprice(mc_url):
    responce = requests.get(mc_url)
    soup = BeautifulSoup(responce.text,'lxml')
    price = soup.find_all('span',{'id':'sp_val'})[0].text
    price = float(price.replace(',',''))
    return price

In [ ]:
def dataframe(df):
    data = []
    for i in range(0,len(df)):
        calloi=callcoi=cltp=putoi=putcoi=pltp=0
        ctotaltradevaluem=ctotalBellq=ctotalSellq=ptotaltradevaluem=ptotalBellq=ptotalSellq=PUT_iv=0
        pbidQty=pbidprice=paskQty=paskPrice=cbidQty=cbidprice=caskQty=caskPrice=CALL_iv = 0
        stp = df['strikePrice'][i]
        if(df['CE'][i]==0):
            calloi = callcoi =0
        else:
            calloi = float(df['CE'][i]['openInterest'])
            callcoi = float(df['CE'][i]['changeinOpenInterest'])
            cltp    = float(df['CE'][i]['lastPrice'])
            ctotaltradevaluem  =float(df['CE'][i]['totalTradedVolume'])
            ctotalBellq = float(df['CE'][i]['totalBuyQuantity'])
            ctotalSellq = float(df['CE'][i]['totalSellQuantity'])
            cbidQty = float(df['CE'][i]['bidQty'])
            cbidprice = float(df['CE'][i]['bidprice'])
            caskQty = float(df['CE'][i]['askQty'])
            caskPrice = float(df['CE'][i]['askPrice'])
            CALL_iv = float(df['CE'][i]['impliedVolatility'])
            # cstp = float(df['CE'][i]['strikePrice'])
        if(df['PE'][i]==0):
            putoi = putcoi = 0
        else:
            putoi = float(df['PE'][i]['openInterest'])
            putcoi = float(df['PE'][i]['changeinOpenInterest'])
            pltp    = float(df['PE'][i]['lastPrice'])
            ptotaltradevaluem  = float(df['PE'][i]['totalTradedVolume'])
            ptotalBellq = float(df['PE'][i]['totalBuyQuantity'])
            ptotalSellq = float(df['PE'][i]['totalSellQuantity'])
            pbidQty = float(df['PE'][i]['bidQty'])
            pbidprice = float(df['PE'][i]['bidprice'])
            paskQty = float(df['PE'][i]['askQty'])
            paskPrice = float(df['PE'][i]['askPrice'])
            PUT_iv = float(df['PE'][i]['impliedVolatility'])
            pstp = float(df['PE'][i]['underlyingValue'])


        opdata = {
            'CALL_TTV':ctotaltradevaluem,'C_BID_Q': cbidQty,'C_BID_P' :cbidprice,'C_ASK_Q' :caskQty,'C_ASK_P' : caskPrice,
            'CALL_OI' : calloi,'CALL_CHNG_OI':callcoi,'total_call_OI':calloi+callcoi,'CALL_IV' : CALL_iv ,
            'CALL_BQ':ctotalBellq,'CALL_SA':ctotalSellq,'spread_call':caskQty-cbidprice,'STRICK_PRICE':stp,'PUT_TTV':ptotaltradevaluem
            ,'P_BID_Q': pbidQty,'P_BID_P' :pbidprice,'P_ASK_Q' :paskQty,'P_ASK_P' : paskPrice,'PUT_IV' : PUT_iv ,
            'PUT_SA':ptotalSellq,'PUT_BQ':ptotalBellq,'PUT_CHNG_OI':putcoi,'PUT_OI' : putoi,'total_put_OI':putcoi+putoi,'spread_put':paskQty-pbidprice,'current_stp':pstp}
        data.append(opdata)
    optionchain = pd.DataFrame(data)
    #optionchain =optionchain.append(optionchain[['CALL_OI','CALL_CHNG_OI','PUT_OI','PUT_CHNG_OI','CALL_TTV','PUT_TTV','CALL_BQ','PUT_BQ','CALL_SA','PUT_SA']].sum(),ignore_index=True).fillna(0)
    return optionchain


In [ ]:
def main(session,url,header):
    request = session.get(url,headers=header).json()['filtered']['data']
    df1 = pd.DataFrame(request).fillna(0)
    option_df = dataframe(df1)
    opdata = {
            'CALL_OI'  : float(option_df['CALL_OI'].sum()),
            'PUT_OI'   : float(option_df['PUT_OI'].sum()),
            'CALL_COI' : float(option_df['CALL_CHNG_OI'].sum()),
            'PUT_COI'  : float(option_df['PUT_CHNG_OI'].sum()),
            # 'CALL_VOL' : float(option_df['CALL_TTV'].sum()),
            # 'PUT_VOL'  : float(option_df['PUT_TTV'].sum()),
            'CALL_TBQ' : float(option_df['CALL_BQ'].sum()),
            'CALL_TSQ' : float(option_df['CALL_SA'].sum()),
            'PUT_TBQ'  : float(option_df['PUT_BQ'].sum()),
            'PUT_TSQ'  : float(option_df['PUT_SA'].sum()),
            # 'OI_DIFF'  : float(option_df['PUT_OI'].sum()/option_df['CALL_OI'].sum()),
            # 'COI_DIFF' : float(option_df['PUT_CHNG_OI'].sum()/option_df['CALL_CHNG_OI'].sum()),
            'VOL_DIFF' : float(option_df['CALL_TTV'].sum()/option_df['PUT_TTV'].sum()),
            'BUY_DIFF': float((option_df['PUT_SA'].sum()+option_df['CALL_BQ'].sum())/(option_df['CALL_SA'].sum()+option_df['PUT_BQ'].sum())),
            'SELL_DIFF': float((option_df['CALL_SA'].sum()+option_df['PUT_BQ'].sum())/(option_df['PUT_SA'].sum()+option_df['CALL_BQ'].sum())),
            # 'CALL_OI_RATIO' : float(option_df['CALL_OI'].sum()/option_df['CALL_CHNG_OI'].sum()),
            # 'PUT_OI_RATIO' : float(option_df['PUT_OI'].sum()/option_df['PUT_CHNG_OI'].sum()),
                                   }
    #print(opdata)
    return opdata,option_df

In [ ]:
#max_CS = max_CB = max_PS = max_PB = max_CT = max_PT = max_COI = max_POI = 0
#max_CS1 = max_CB1 = max_PS1 = max_PB1 = max_CT1 = max_PT1 = max_COI1 = max_POI1 = 0

In [ ]:
def reurn_max(x,option_df,n):

    return   (sqldf('''SELECT STRICK_PRICE  \
                            FROM \
                                    (SELECT STRICK_PRICE  , {x},row_number()OVER (PARTITION BY STRICK_PRICE order by {x} DESC)\
                                    AS rownumber FROM option_df)\
                        WHERE rownumber = 1 order by {x} DESC LIMIT 5'''.format(x=x)).values)[n]

In [ ]:
def max_dict(option_df,mc_url):
  max_dic = {
                'max_CS'   : [
                    reurn_max('CALL_SA',option_df,0)[0],
                    reurn_max('CALL_SA',option_df,1)[0],
                    reurn_max('CALL_SA',option_df,2)[0]
                ],
                'max_CB'   : [
                    reurn_max('CALL_BQ',option_df,0)[0],
                    reurn_max('CALL_BQ',option_df,1)[0],
                    reurn_max('CALL_BQ',option_df,2)[0]
                ],
                'max_CT'   : [
                    reurn_max('CALL_TTV',option_df,0)[0],
                    reurn_max('CALL_TTV',option_df,1)[0],
                    reurn_max('CALL_TTV',option_df,2)[0]
                ],
                 'max_CTOI'   : [
                    reurn_max('total_call_OI',option_df,0)[0],
                    reurn_max('total_call_OI',option_df,1)[0],
                    reurn_max('total_call_OI',option_df,2)[0]
                ],
                'max_COI'  : [
                    reurn_max('CALL_OI',option_df,0)[0],
                    reurn_max('CALL_OI',option_df,1)[0],
                    reurn_max('CALL_OI',option_df,2)[0]
                ],
                'max_CCOI' : [
                    reurn_max('CALL_CHNG_OI',option_df,0)[0],
                    reurn_max('CALL_CHNG_OI',option_df,1)[0],
                    reurn_max('CALL_CHNG_OI',option_df,2)[0]
                ],
                'STP'      : [int(MC_strickprice(mc_url)),int(MC_strickprice(mc_url)),int(MC_strickprice(mc_url))],

                'max_PCOI' : [
                    reurn_max('PUT_CHNG_OI',option_df,0)[0],
                    reurn_max('PUT_CHNG_OI',option_df,1)[0],
                    reurn_max('PUT_CHNG_OI',option_df,2)[0]
                ],
                'max_POI'  : [
                    reurn_max('PUT_OI',option_df,0)[0],
                    reurn_max('PUT_OI',option_df,1)[0],
                    reurn_max('PUT_OI',option_df,2)[0]
                ],
                'max_PTOI'   : [
                    reurn_max('total_put_OI',option_df,0)[0],
                    reurn_max('total_put_OI',option_df,1)[0],
                    reurn_max('total_put_OI',option_df,2)[0]
                ],
                'max_PT'   : [
                    reurn_max('PUT_TTV',option_df,0)[0],
                    reurn_max('PUT_TTV',option_df,1)[0],
                    reurn_max('PUT_TTV',option_df,2)[0]
                ],
                'max_PB'   : [
                    reurn_max('PUT_BQ',option_df,0)[0],
                    reurn_max('PUT_BQ',option_df,1)[0],
                    reurn_max('PUT_BQ',option_df,2)[0]
                ],
                'max_PS'   : [
                    reurn_max('PUT_SA',option_df,0)[0],
                    reurn_max('PUT_SA',option_df,1)[0],
                    reurn_max('PUT_SA',option_df,2)[0]
                    ]
                }
  return max_dic

In [1]:
def analyze_options_with_buyers_sellers(df):
    results = []
    
    for index, row in df.iterrows():
        # Extract all relevant columns
        call_iv = row['CALL_IV']
        put_iv = row['PUT_IV']
        call_oi = row['CALL_OI']
        put_oi = row['PUT_OI']
        call_change_oi = row['CALL_CHNG_OI']
        put_change_oi = row['PUT_CHNG_OI']
        current_price = row['current_stp']
        strike_price = row['STRICK_PRICE']
        call_bid = row['C_BID_P']
        call_ask = row['C_ASK_P']
        put_bid = row['P_BID_P']
        put_ask = row['P_ASK_P']
        spread_call = row['spread_call']
        spread_put = row['spread_put']
        call_buyers = row["CALL_BQ"]
        call_sellers = row["CALL_SA"]
        put_buyers = row["PUT_BQ"]
        put_sellers = row["PUT_SA"]

        # Calculate the Put-Call Ratio
        put_call_ratio = put_oi / call_oi if call_oi != 0 else float('inf')

        # Calculate if the option is ITM/ATM/OTM
        atm_threshold = 0.01  # ATM if within 1% of current price
        if call_iv > put_iv and call_change_oi > 0 and spread_call < 20 and call_buyers > call_sellers :
            action = 'Buy Call'
            price_to_buy = call_ask
        elif put_iv > call_iv and put_change_oi > 0 and spread_put < 20 and put_buyers > put_sellers :
            action = 'Buy Put'
            price_to_buy = put_ask
        else:
            action = 'No Action'
            price_to_buy = None

        # Compile the results for each row of data
        results.append({
            'Action': action,
            'PTB': price_to_buy,
            'Call_IV': call_iv,
            'Call_OI': call_oi,
            'Call_COI': call_change_oi,
            'Call_BQ': call_buyers,
            'Call_SQ': call_sellers,
            'Current Price': current_price,
            'STP': strike_price,
            'Put_SQ': put_sellers,
            'Put_BQ': put_buyers,
            'Put_COI': put_change_oi,
            'Put_OI': put_oi,
            'Put_IV': put_iv,
            # 'Option Type': option_type,
            'Put-Call Ratio': put_call_ratio
        })
    
    return pd.DataFrame(results)

In [ ]:
def generate_signals(data):
    # Initialize signal column
    data['Signal'] = 0
    
    # Calculate PUT and CALL signals
    put_condition = (data['PUT_TSQ'] > data['PUT_TSQ'].shift()) & (data['PUT_COI'] > data['PUT_COI'].shift())
    call_condition = (data['CALL_TSQ'] > data['CALL_TSQ'].shift()) & (data['CALL_COI'] > data['CALL_COI'].shift())
    
    # Set signal values
    data.loc[put_condition, 'Signal'] = 1  # Buy PUT signal
    data.loc[call_condition, 'Signal'] = -1  # Buy CALL signal
    
    return data

In [ ]:
def run_main(option_dataframe,max_df,session,url,header,mc_url):
    while True:
        clear_output(wait=True)
        option_dataframe,option_df=option_dataframe._append(main(session,url,header)[0],ignore_index=True),main(session,url,header)[1]
        option_dataframe.drop_duplicates(inplace = True)
        analysis_results = analyze_options_with_buyers_sellers(option_df)
        price = int(MC_strickprice(mc_url))
        max_dic = max_dict(option_df,mc_url)
        range_price = list(range(round(price/100)*100-300,round(price/100)*100+300,50))
        max_dic = max_dict(option_df,mc_url)
        max_df = max_df._append(pd.DataFrame(max_dic))
        display_df = option_df[['CALL_IV','spread_call','CALL_OI','CALL_CHNG_OI','CALL_BQ','CALL_SA','STRICK_PRICE',
                                'PUT_SA','PUT_BQ','PUT_CHNG_OI','PUT_OI','spread_put','PUT_IV']]
        short_writer = ((display_df['CALL_CHNG_OI'] > 0) & (display_df['PUT_CHNG_OI'] > 0))
        short_cover = ((display_df['CALL_CHNG_OI'] < 0) | (display_df['PUT_CHNG_OI'] < 0))
        long_cover = ((display_df['CALL_CHNG_OI'] > 0) & (display_df['PUT_CHNG_OI'] < 0))
        long_writer = ((display_df['CALL_CHNG_OI'] > 0) & (display_df['PUT_CHNG_OI'] > 0))
        display_df.loc[:, 'Scenario'] = 'None'   # Default scenario
        display_df.loc[short_writer, 'Scenario'] = 'S-Writer'
        display_df.loc[short_cover, 'Scenario'] = 'S-Cover'
        display_df.loc[long_cover, 'Scenario'] = 'L-Cover'
        display_df.loc[long_writer, 'Scenario'] = 'L-Writer'
        option_dataframe = generate_signals(option_dataframe)
        display(option_dataframe.tail(5))
        display(max_df.tail(3)
        # ,max_df_2.tail(3)
        ,analysis_results[(analysis_results.STP.isin(range_price))]
        # ,display_df[(display_df.STRICK_PRICE.isin(range_price))]
        )
        # display(option_dataframe.tail(3))
        time.sleep(2)

